# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [103]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gk

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [104]:
path = "C:/Users/chadi/Desktop/Instructions/starter_code/weather_info.csv"
df = pd.read_csv(path)
df.set_index("Index")

,Country,Cities,DateTime,Latitude,Longitude,Weather,Humidity,Temperature,Max Temperature,Cloudiness,Wind Speed
Index,,,,,,,,,,,
0,NZ,Te Anau,1657450094,-45.4167,167.7167,scattered clouds,94,-0.54,-0.54,39,0.84
1,KY,Bodden Town,1657450094,19.2833,-81.2500,scattered clouds,81,26.11,27.74,37,1.34
2,AR,Ushuaia,1657449879,-54.8000,-68.3000,light snow,86,-0.19,-0.19,100,9.77
3,EC,Puerto Ayora,1657450094,-0.7393,-90.3518,overcast clouds,96,20.31,20.31,90,3.13
4,SA,Riyadh,1657449684,24.6877,46.7219,overcast clouds,6,43.08,43.08,100,6.70
...,...,...,...,...,...,...,...,...,...,...,...
564,GL,Maniitsoq,1657450141,65.4167,-52.9000,overcast clouds,90,4.01,4.01,100,2.49
565,IR,Dārāb,1657450141,28.7519,54.5444,clear sky,21,40.00,40.00,9,4.87
566,ID,Tambun,1657450141,-6.2641,107.0712,broken clouds,68,28.94,28.94,75,3.19


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [105]:
import gmaps
gmaps.configure(api_key = gk)
locations = df[['Latitude', 'Longitude']]
weights = df['Humidity']
fig = gmaps.figure(map_type='HYBRID')
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [106]:
dfm = df.loc[df["Weather"].isnull()]
cdf = df[~ df["Humidity"].isin(dfm)]
cdf.set_index("Index")

,Country,Cities,DateTime,Latitude,Longitude,Weather,Humidity,Temperature,Max Temperature,Cloudiness,Wind Speed
Index,,,,,,,,,,,
0,NZ,Te Anau,1657450094,-45.4167,167.7167,scattered clouds,94,-0.54,-0.54,39,0.84
1,KY,Bodden Town,1657450094,19.2833,-81.2500,scattered clouds,81,26.11,27.74,37,1.34
2,AR,Ushuaia,1657449879,-54.8000,-68.3000,light snow,86,-0.19,-0.19,100,9.77
3,EC,Puerto Ayora,1657450094,-0.7393,-90.3518,overcast clouds,96,20.31,20.31,90,3.13
4,SA,Riyadh,1657449684,24.6877,46.7219,overcast clouds,6,43.08,43.08,100,6.70
...,...,...,...,...,...,...,...,...,...,...,...
564,GL,Maniitsoq,1657450141,65.4167,-52.9000,overcast clouds,90,4.01,4.01,100,2.49
565,IR,Dārāb,1657450141,28.7519,54.5444,clear sky,21,40.00,40.00,9,4.87
566,ID,Tambun,1657450141,-6.2641,107.0712,broken clouds,68,28.94,28.94,75,3.19


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [111]:
# Creating a vacation preference
ndf = (df.loc[(df["Temperature"] < 46)  & (df["Temperature"] > 33) & (df["Wind Speed"] < 6) & (df["Cloudiness"] < 40)])
ndf

,Index,Country,Cities,DateTime,Latitude,Longitude,Weather,Humidity,Temperature,Max Temperature,Cloudiness,Wind Speed
18,18,ML,Tessalit,1657450096,20.1986,1.0114,clear sky,9,39.69,39.69,5,2.11
43,43,ML,Taoudenni,1657450098,22.6783,-3.9836,scattered clouds,6,44.04,44.04,30,2.02
47,47,TM,Balkanabat,1657450098,39.5108,54.3671,clear sky,12,38.30,38.30,0,4.35
80,80,IN,Karīmganj,1657450101,24.8667,92.3500,few clouds,70,33.22,33.22,12,1.56
237,237,IR,Tabas,1657450113,33.5959,56.9244,clear sky,6,42.07,42.07,2,0.52
250,250,SA,Turaif,1657450114,31.6725,38.6637,clear sky,18,34.14,34.14,0,4.12
317,317,MR,Adrar,1657450119,20.5022,-10.0711,clear sky,10,40.34,40.34,0,2.47
322,322,EG,Suez,1657450120,29.9737,32.5263,clear sky,28,35.54,35.54,0,4.35
330,330,KZ,Arys,1657450120,42.4252,68.7950,few clouds,12,36.14,36.14,20,5.48
351,351,KZ,Beyneu,1657450122,45.3167,55.2000,clear sky,16,35.15,35.15,3,2.01


In [121]:
hotel_name = []
hotel_lat = []
hotel_lng = []

nwdf = ndf[['Latitude', 'Longitude']]
for x in nwdf.iterrows():
    try:
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?keyword=Hotel&location={x[1]['Latitude']},{x[1]['Longitude']}&radius=5000&type=Hotel&key={gk}"
        data = requests.get(url).json()
        hotel_name.append(data['results'][0]['name'])
        hotel_lat.append(x[1]['Latitude'])
        hotel_lng.append(x[1]['Longitude'])
    except:
        print(f'No Hotels found for country')


No Hotels found for country
No Hotels found for country
No Hotels found for country
No Hotels found for country
No Hotels found for country
No Hotels found for country
No Hotels found for country


In [143]:
hoteldf = pd.DataFrame({'Hotel Name': hotel_name, 'Latitude': hotel_lat,'Longitude': hotel_lng})
mdf = pd.merge(ndf, hoteldf, on = 'Latitude', how = 'inner')
mdf.rename(columns = {'Cities':'City', 'Longitude_x':'Longitude'}, inplace = True)
dft = mdf[['Country', 'City', 'Hotel Name', 'Latitude', 'Longitude', 'Humidity', 'Weather']]
dft

,Country,City,Hotel Name,Latitude,Longitude,Humidity,Weather
0,TM,Balkanabat,Nebitchi Hotel,39.5108,54.3671,12,clear sky
1,IN,Karīmganj,Hotel Nakshatra,24.8667,92.3500,70,few clouds
2,IR,Tabas,Amir hotel,33.5959,56.9244,6,clear sky
3,SA,Turaif,Le Park Concord Turayf,31.6725,38.6637,18,clear sky
4,EG,Suez,Red Sea Hotel,29.9737,32.5263,28,clear sky
5,KZ,Arys,"Gostinitsa ""Madani""",42.4252,68.7950,12,few clouds
6,KZ,Beyneu,Gostinitsa Zhibek-Zholy Beyneu,45.3167,55.2000,16,clear sky
7,IQ,Dahuk,Dilshad Palace,36.8695,42.9940,12,clear sky
8,UZ,Yangi Marg`ilon,TANTANA HOTEL FERGANA,40.4272,71.7189,15,scattered clouds
9,MR,Atar,les toiles maures,20.5169,-13.0499,23,clear sky


In [144]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Weather}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in dft.iterrows()]
hlocations = dft[["Latitude", "Longitude"]]

In [145]:
# Add marker layer ontop of heat map
fig1 = fig
markers = gmaps.marker_layer(hlocations, info_box_content = hotel_info)
fig1.add_layer(markers)

# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))